# Análise de qualidade dos dados
Aqui faremos a etapa de análise de qualidade de cada atributo do conjunto de dados. Faremos isso utilizando consultas SQL baseadas nas características de cada atributo registradas no catálogo de dados.

Antes deste, foi feito o processo de ETL, que pode ser visto no link a seguir.

https://github.com/samuelrubert/pucrio-mvp-engenhariadados/blob/main/Notebooks/CCUS_etapa1_ETL.ipynb

Passemos à análise dos atributos.

## 1. Project_name
O nome do projeto é um texto livre, porém não pode ser vazio.

Vamos verificar se há linhas em que o nome do projeto seja nulo.

In [0]:
%sql
SELECT count(1) AS `Nomes de projeto vazios`
FROM dt0051_dev.treinamento.iea_ccus_projects
WHERE Project_name IS NULL

Não há campos vazios. Dessa forma, não se verificou problemas na coluna.

## 2. ID
O processo de ETL já assegurou que o ID seja sempre um número inteiro.

Além disso, ele deve ser único. Vamos verificar.

In [0]:
%sql
SELECT ID AS `ID repetido`, count(ID) AS Quantidade
FROM dt0051_dev.treinamento.iea_ccus_projects
GROUP BY ID
HAVING count(ID) > 1

Encontramos três IDs que se repetem na tabela. Vamos olhar para esses dados.

In [0]:
%sql
SELECT *
FROM dt0051_dev.treinamento.iea_ccus_projects
WHERE ID IN (1103, 365, 863)
ORDER BY ID ASC

Vamos que os dados não são idênticos, porém a similaridade dos dados indica que alguns dos elementos duplicados possam estar relacionados, ou até mesmo ser uma continuação do projeto.

Não conhecemos plenamente a semântica da repetição do ID, por isso não alteraremos os dados.

## 3. Country
O valor do campo pode ser um texto livre indicando um ou mais países ou regiões. Vamos apenas verificar se há linhas vazias.

In [0]:
%sql
SELECT count(1) as `Países vazios`
FROM dt0051_dev.treinamento.iea_ccus_projects
WHERE Country IS NULL

Não há campos vazios. Dessa forma, não se verificou problemas na coluna.

## 4. Partners
O valor do campo pode ser um texto livre indicando as empresas envolvidas no projeto. Vamos apenas verificar se há linhas vazias.

In [0]:
%sql
SELECT count(1) as `Parceiros vazios`
FROM dt0051_dev.treinamento.iea_ccus_projects
WHERE Partners IS NULL

Não há campos vazios. Dessa forma, não se verificou problemas na coluna.

## 5. Project_type
O tipo de projeto deve ser um dos valores categóricos permitidos. Vamos verificar.

In [0]:
%sql
SELECT Project_type, count(1) AS Quantidade
FROM dt0051_dev.treinamento.iea_ccus_projects
WHERE Project_type NOT IN ('Full chain', 'Capture', 'Transport', 'Storage', 'T&S', 'CCU')
GROUP BY Project_type

Identificamos oito registros em que o tipo "Full Chain" foi preenchido de forma irregular, com o C maiúsculo. Vamos corrigir esses valores.

In [0]:
%sql
UPDATE dt0051_dev.treinamento.iea_ccus_projects
SET Project_type = 'Full chain'
WHERE Project_type == 'Full Chain'

A saída mostra que oito linhas foram alteradas. Vamos refazer a consulta para nos certificar de que o problema foi corrigido.

In [0]:
%sql
SELECT Project_type, count(1) AS Quantidade
FROM dt0051_dev.treinamento.iea_ccus_projects
WHERE Project_type NOT IN ('Full chain', 'Capture', 'Transport', 'Storage', 'T&S', 'CCU')
GROUP BY Project_type

Nenhum resultado foi retornado. Sendo assim, não há mais nenhum problema com a coluna Project_type.

## 6. Announcement
O processo de ETL já assegurou que o valor desta coluna seja sempre um número inteiro.

Ele deve ser também um ano válido, não pode ser futuro, e não pode ser num passado muito distante. Vamos verificar esses limites, escolhendo como margem mínima para a avaliação o ano de 1950, época em que a indústria de petróleo ainda começava a crescer mundialmente.

In [0]:
%sql
SELECT count(1) AS `Datas de anúncio fora da margem`
FROM dt0051_dev.treinamento.iea_ccus_projects
WHERE Announcement < 1950 OR Announcement > 2024

Nenhuma data foi encontrada fora da margem definida. Com isso, não há problemas de qualidade a resolver.

## 7. FID
O processo de ETL já assegurou que o valor desta coluna seja sempre um número inteiro.

A data pode ser futura, porém não pode ser um futuro muito distante. Daremos uma margem de 100 anos no futuro. Além disso, não faz sentido que seja anterior à data de anúncio do projeto.

In [0]:
%sql
SELECT count(1) AS `FIDs fora da margem`
FROM dt0051_dev.treinamento.iea_ccus_projects
WHERE FID > 2124 OR FID < Announcement

Nenhuma data foi encontrada fora da margem definida. Com isso, não há problemas de qualidade a resolver.

## 8. Operation
O processo de ETL já assegurou que o valor desta coluna seja sempre um número inteiro.

A data pode ser futura, porém não pode ser um futuro muito distante. Daremos uma margem de 100 anos no futuro. Além disso, não faz sentido que seja anterior à data final de investimento do projeto.

In [0]:
%sql
SELECT count(1) AS `Datas de operação fora da margem`
FROM dt0051_dev.treinamento.iea_ccus_projects
WHERE Operation > 2124 OR Operation < FID

Nenhuma data foi encontrada fora da margem definida. Com isso, não há problemas de qualidade a resolver.

## 9. Suspension_decommissioning
O processo de ETL já assegurou que o valor desta coluna seja sempre um número inteiro.

A data pode ser futura, porém não pode ser um futuro muito distante. Daremos uma margem de 100 anos no futuro. Além disso, não faz sentido que seja anterior à data de início de operação do projeto.

In [0]:
%sql
SELECT count(1) AS `Datas de descomissionamento fora da margem`
FROM dt0051_dev.treinamento.iea_ccus_projects
WHERE Suspension_decommissioning > 2124 OR Suspension_decommissioning < Operation

Nenhuma data foi encontrada fora da margem definida. Com isso, não há problemas de qualidade a resolver.

## 10. Project_Status
O status do projeto deve ser um dos valores categóricos permitidos. Vamos verificar.

In [0]:
%sql
SELECT Project_Status, count(1) AS `Quantidade`
FROM dt0051_dev.treinamento.iea_ccus_projects
WHERE Project_Status NOT IN ('Planned', 'Under construction', 'Operational', 'Suspended', 'Decommissioned')
GROUP BY Project_Status

Foram encontrados 5 projetos com o status "Suspended/cancelled/decommissioned". Neste caso, como não sabemos qual seria o status correto, não é possível corrigir.

## 11. Project_phase
O processo de ETL já assegurou que o valor desta coluna seja sempre um número inteiro.

Conforme definido no catálogo, deve ser no mínimo 1. Vamos verificar.

In [0]:
%sql
SELECT Project_phase, count(1) AS `Quantidade`
FROM dt0051_dev.treinamento.iea_ccus_projects
WHERE Project_phase < 1
GROUP BY Project_phase

Não encontramos problemas nesta coluna.

## 12. Announced_capacity_low_Mt_CO2_yr
O processo de ETL já assegurou que o valor desta coluna seja sempre um número decimal.

Vamos verificar se há valores negativos.

In [0]:
%sql
SELECT ID, Announced_capacity_low_Mt_CO2_yr
FROM dt0051_dev.treinamento.iea_ccus_projects
WHERE Announced_capacity_low_Mt_CO2_yr < 0

Não encontramos problemas nesta coluna.

## 13. Announced_capacity_high_Mt_CO2_yr
O processo de ETL já assegurou que o valor desta coluna seja sempre um número decimal.

Vamos verificar se há valores negativos.

In [0]:
%sql
SELECT ID, Announced_capacity_high_Mt_CO2_yr
FROM dt0051_dev.treinamento.iea_ccus_projects
WHERE Announced_capacity_high_Mt_CO2_yr < 0

Não encontramos problemas nesta coluna.

## 14. Estimated_capacity_by_IEA_Mt_CO2_yr
O processo de ETL já assegurou que o valor desta coluna seja sempre um número decimal.

Vamos verificar se há valores negativos.

In [0]:
%sql
SELECT ID, Estimated_capacity_by_IEA_Mt_CO2_yr
FROM dt0051_dev.treinamento.iea_ccus_projects
WHERE Estimated_capacity_by_IEA_Mt_CO2_yr < 0

Não encontramos problemas nesta coluna.

## 15. Sector
O setor do projeto deve ser um dos valores categóricos permitidos. Vamos verificar.

In [0]:
%sql
SELECT Sector, count(1) AS Quantidade
FROM dt0051_dev.treinamento.iea_ccus_projects
WHERE Sector NOT IN ('Power and heat', 'Natural gas processing/LNG', 'Biofuels', 'Hydrogen or ammonia', 'Other fuel transformation', 'Cement', 'Iron and steel', 'Chemicals', 'Other industry', 'DAC', 'Transport', 'Storage', 'T&S')
GROUP BY Sector

Não encontramos problemas nesta coluna.

## 16. Fate_of_carbon
O destino do carbono deve ser um dos valores categóricos permitidos. Vamos verificar.

In [0]:
%sql
SELECT Fate_of_carbon, count(1) AS Quantidade
FROM dt0051_dev.treinamento.iea_ccus_projects
WHERE Fate_of_carbon NOT IN ('Dedicated storage', 'EOR', 'Use', 'Unknown/unspecified')
GROUP BY Fate_of_carbon

Encontramos um novo valor, "Mixed", em alguns registros. Não há como saber se algum valor do catálogo seria adequado para substituição, portanto não é possível resolver.

## 17. Part_of_CCUS_hub
Este campo é um texto livre e não obrigatório. Vamos apenas descobrir qual fração dos valores é vazia.

In [0]:
%sql
SELECT (count(1) - count(Part_of_CCUS_hub))/count(1)*100 AS `Part_of_CCUS_hub %vazio`
FROM dt0051_dev.treinamento.iea_ccus_projects

Vimos que quase metade dos valores estão vazios. Porém, não há problemas de qualidade a resolver.

## 18. Region
Este campo é um texto livre. Vamos descobrir se há valores vazios.

In [0]:
%sql
SELECT ID, Region
FROM dt0051_dev.treinamento.iea_ccus_projects
WHERE Region IS NULL

Não há valores vazios na coluna.

## 19. Ref_1
A referência deve ser um URL válido. Vamos avaliar esse requisito verificando se há valores que não começam com "http".

In [0]:
%sql
SELECT Ref_1
FROM dt0051_dev.treinamento.iea_ccus_projects
WHERE Ref_1 NOT LIKE 'http%'

Encontramos um valor que não é um URL válido.

Pesquisando essa referência na internet, foi possível obter o link correto. Vamos inseri-lo no campo.

In [0]:
%sql
UPDATE dt0051_dev.treinamento.iea_ccus_projects
SET Ref_1 = 'https://www.nstauthority.co.uk/news-publications/net-zero-boost-as-carbon-storage-licences-accepted/'
WHERE Ref_1 == 'North Sea Transition Authority (NSTA): Net zero boost as carbon storage licences accepted - 2023 - News - News & <br/>publications (nstauthority.co.uk)'

Agora vamos refazer a consulta para certificar que não haja mais problemas.

In [0]:
%sql
SELECT Ref_1
FROM dt0051_dev.treinamento.iea_ccus_projects
WHERE Ref_1 NOT LIKE 'http%'

Não há mais problemas de qualidade nesta coluna.

## 20. Ref_2
A referência deve ser um URL válido. Vamos avaliar esse requisito verificando se há valores que não começam com "http".

In [0]:
%sql
SELECT Ref_2
FROM dt0051_dev.treinamento.iea_ccus_projects
WHERE Ref_2 NOT LIKE 'http%'

Os dois primeiros valores são referências a arquivos PDF, aparentemente num endereço local acessível ao autor da planilha. Não há como resolver.

No terceiro caso, foi possível obter o link correto. Vamos inseri-lo.

In [0]:
%sql
UPDATE dt0051_dev.treinamento.iea_ccus_projects
SET Ref_2 = 'https://www.marketscreener.com/quote/stock/JAPAN-PETROLEUM-EXPLORATI-6496655/news/Japan-Petroleum-Exploration-Entrust-a-Study-in-East-Niigata-Area-as-the-Business-Feasibility-St-44542777/'
WHERE Ref_2 == 'Japan Petroleum Exploration : Entrust a Study in East-Niigata Area as the “Business Feasibility Study on Japanese Advanced CCS Project” in FY2023 by JOGMEC -August 08, 2023 at 01:06 am EDT | MarketScreener'

Agora vamos refazer a consulta para certificar que não haja outros problemas.

In [0]:
%sql
SELECT Ref_2
FROM dt0051_dev.treinamento.iea_ccus_projects
WHERE Ref_2 NOT LIKE 'http%'

Não há mais problemas de qualidade que possam ser corrigidos nesta coluna.

## 21. Ref_3
A referência deve ser um URL válido. Vamos avaliar esse requisito verificando se há valores que não começam com "http".

In [0]:
%sql
SELECT Ref_3
FROM dt0051_dev.treinamento.iea_ccus_projects
WHERE Ref_3 NOT LIKE 'http%'

Não há problemas de qualidade nesta coluna.

## 22. Ref_4
A referência deve ser um URL válido. Vamos avaliar esse requisito verificando se há valores que não começam com "http".

In [0]:
%sql
SELECT Ref_4
FROM dt0051_dev.treinamento.iea_ccus_projects
WHERE Ref_4 NOT LIKE 'http%'

Não há problemas de qualidade nesta coluna.

## 23. Ref_5
A referência deve ser um URL válido. Vamos avaliar esse requisito verificando se há valores que não começam com "http".

In [0]:
%sql
SELECT Ref_5
FROM dt0051_dev.treinamento.iea_ccus_projects
WHERE Ref_5 NOT LIKE 'http%'

Não há problemas de qualidade nesta coluna.

## 24. Ref_6
A referência deve ser um URL válido. Vamos avaliar esse requisito verificando se há valores que não começam com "http".

In [0]:
%sql
SELECT Ref_6
FROM dt0051_dev.treinamento.iea_ccus_projects
WHERE Ref_6 NOT LIKE 'http%'

Não há problemas de qualidade nesta coluna.

## 25. Ref_7
A referência deve ser um URL válido. Vamos avaliar esse requisito verificando se há valores que não começam com "http".

In [0]:
%sql
SELECT Ref_7
FROM dt0051_dev.treinamento.iea_ccus_projects
WHERE Ref_7 NOT LIKE 'http%'

Não há problemas de qualidade nesta coluna.

# Próxima etapa
Finalizada a etapa de análise de qualidade, partiremos para a solução do problema.

Acesse o notebook correspondente no link a seguir.

https://github.com/samuelrubert/pucrio-mvp-engenhariadados/blob/main/Notebooks/CCUS_etapa3_solucao.ipynb